In [1]:
# Parameters
msgs = "Ran from Airflow at 2021-06-24T21:39:49.962170+00:00!"


In [2]:
import pandas as pd
import mysql.connector
import os
#import json
import datetime

In [3]:
with open("/home/kali/project/input_config.txt") as f:
    input_file=f.readline()
print("will process:",input_file)

will process: /home/kali/project/input/meetingAttendanceReport(General) (9).csv


In [4]:
meta_data=[]
log_call=[]
lines=[]
with open(input_file,encoding="utf-16") as f:
    lines=f.readlines()
meta_data=lines[0:5]
log_call=lines[6:]

In [5]:
course = meta_data[2].replace("Titre de la réunion\t","").replace("\n","")
course

'General'

In [6]:
with open(f"/home/kali/project/output/log_{input_file.split('/')[-1]}","w") as f:
    for value in log_call:
        f.write(value)
    f.close()
df_log = pd.read_csv(f"/home/kali/project/output/log_{input_file.split('/')[-1]}",sep="\t")
df_log

,Nom complet,Heure d'arrivée,Heure de départ,Durée,E-mail,Rôle
0,Imade BENELALLAM,14/04/2021 à 13:08:06,14/04/2021 à 15:35:57,2 h 27 m,i.benelallam@insea.ac.ma,Organisateur
1,Andre SINARE,14/04/2021 à 13:08:18,14/04/2021 à 14:42:38,1 h 34 m,asinare@insea.ac.ma,Présentateur
2,Chaimaa BELKARKOR,14/04/2021 à 13:08:25,14/04/2021 à 15:28:57,2 h 20 m,cbelkarkor@insea.ac.ma,Présentateur
3,Hibatallah BOUNAR,14/04/2021 à 13:08:51,14/04/2021 à 14:42:55,1 h 34 m,hbounar@insea.ac.ma,Présentateur
4,Mahamadou SANGARE,14/04/2021 à 13:09:08,14/04/2021 à 14:47:29,1 h 38 m,msangare@insea.ac.ma,Présentateur
...,...,...,...,...,...,...
82,Amine AJARRAY,14/04/2021 à 15:05:33,14/04/2021 à 15:06:58,1 m 24 s,aajarray@insea.ac.ma,Présentateur
83,Hajar CHAFAI,14/04/2021 à 14:17:22,14/04/2021 à 15:07:42,50 m 20 s,hchafai@insea.ac.ma,Présentateur
84,Yassine ZOUHRI,14/04/2021 à 14:27:34,14/04/2021 à 14:43:17,15 m 42 s,yzouhri@insea.ac.ma,Présentateur
85,Abderrahim ELBAHJI,14/04/2021 à 14:37:19,14/04/2021 à 14:42:58,5 m 39 s,aelbahji@insea.ac.ma,Présentateur


In [7]:
start_str = meta_data[3].replace("Heure de début de la réunion\t","").replace("\n","").replace(" à","")
end_str   = meta_data[4].replace("Heure de fin\t","").replace("\n","").replace(" à","")
start_course = datetime.datetime.strptime(start_str, '%d/%m/%Y %H:%M:%S')
end_course = datetime.datetime.strptime(end_str, '%d/%m/%Y %H:%M:%S')

In [8]:
df_ds = pd.read_excel("/home/kali/project/lists_excel/Liste des 2èmes années DS.xlsx")
#df_dse = pd.read_excel("/home/kali/notebook/project2/lists_excel/Liste 2A DSE.xlsx")

In [9]:
mydb = mysql.connector.connect(database="airflow_db", user="db_kali", password="123456789", host="localhost")

In [10]:
#mycursor = mydb.cursor()
#table="fact_table"
#mycursor.execute(f"SELECT * FROM information_schema.columns WHERE table_schema = 'biabsence' AND table_name   = '{table}' ;")
#myresult = mycursor.fetchall()
#for value in myresult:
#    print(value[3],":",value[7])

In [11]:
def add_student(name,gender,email,course,major):
    mydb = mysql.connector.connect(database="airflow_db", user="db_kali", password="123456789", host="localhost") 
    mycursor = mydb.cursor()
    sql1 = "SELECT * FROM students where email = %s;"
    vals1 = (email,)
    mycursor.execute(sql1, vals1)
    myresult = mycursor.fetchall()
    if len(myresult) != 0:
        return myresult[0][0]
    if len(myresult) == 0:
        sql = "INSERT INTO students (name,gender,email,course,major) VALUES (%s,%s,%s,%s,%s);"
        val = (name,gender,email,course,major)
        mycursor.execute(sql, val)
        mydb.commit()
        # print(mycursor.rowcount, "record inserted.")
        sql3 = "SELECT * FROM students where email = %s;"
        vals3 = (email,)
        mycursor.execute(sql3, vals3)
        myresult = mycursor.fetchall()
        return myresult[0][0]
    
def add_professor(name,email):
    mydb = mysql.connector.connect(database="airflow_db", user="db_kali", password="123456789", host="localhost") 
    mycursor = mydb.cursor()
    sql1 = "SELECT * FROM professors where email = %s;"
    vals1 = (email, )
    mycursor.execute(sql1, vals1)
    myresult = mycursor.fetchall()
    if len(myresult) != 0:
        return myresult[0][0]
    if len(myresult) == 0:
        sql = "INSERT INTO professors (name,email) VALUES (%s,%s);"
        val = (name,email,)
        mycursor.execute(sql, val)
        mydb.commit()
        # print(mycursor.rowcount, "record inserted.")
        sql3 = "SELECT * FROM professors where email = %s;"
        vals3 = (email, )
        mycursor.execute(sql1, vals1)
        myresult = mycursor.fetchall()
        return myresult[0][0]
    
def add_time(hour,day,month,year,date):
    mydb = mysql.connector.connect(database="airflow_db", user="db_kali", password="123456789", host="localhost")
    mycursor = mydb.cursor()
    sql1 = "SELECT * FROM time WHERE year = %s and month=%s and day=%s and hour=%s;"
    vals1 = (year, month, day, hour, )
    mycursor.execute(sql1, vals1)
    myresult = mycursor.fetchall()
    if len(myresult) != 0:
        return myresult[0][0]
    if len(myresult) == 0:
        sql = "INSERT INTO time (hour,day,month,year,date) VALUES (%s,%s,%s,%s,%s);"
        val = (hour,day,month,year,date)
        mycursor.execute(sql, val)
        mydb.commit()
        # print(mycursor.rowcount, "record inserted.")
        sql3 = "SELECT * FROM time WHERE year = %s and month=%s and day=%s and hour=%s;"
        vals3 = (year, month, day, hour, )
        mycursor.execute(sql3, vals3)
        myresult = mycursor.fetchall()
        return myresult[0][0]

In [12]:
hour = start_course.hour
day  = start_course.day
month= start_course.month
year = start_course.year
date = start_course.strftime("%Y-%m-%d %H:%M:%S")
id_time=add_time(hour,day,month,year,date)
id_time

6

In [13]:
#ADD DS TO STUDENTS TABLE
for index, row in df_ds.iterrows():
    major="DS"
    name,gender,email,course,major = row["Nom"],None,row["email"],course,major
    id_student=add_student(name,gender,email,course,major)
    print("student:",id_student)

student: 193
student: 194
student: 195
student: 196
student: 197
student: 198
student: 199
student: 200
student: 201
student: 202
student: 203
student: 204
student: 205
student: 206
student: 207
student: 208
student: 209
student: 210
student: 211
student: 212
student: 213
student: 214
student: 215
student: 216
student: 217
student: 218
student: 219
student: 220
student: 221
student: 222
student: 223
student: 224


In [14]:
#ADD STUDENTS FROM LOG TO TABLE
for index, row in df_log.iterrows():
    if row["Rôle"]!="Organisateur":
        result = df_ds[df_ds["email"]==row["E-mail"]]
        if(len(result) > 0):
            major="DS"
        else:
            major="DSE"
        name,gender,email,course,major = row["Nom complet"],None,row["E-mail"],course,major
        id_student=add_student(name,gender,email,course,major)
        print("student:",id_student)
    else:
        name,email = row["Nom complet"],row["E-mail"]
        id_prof = add_professor(name,email)
        print("professor:",id_prof)

professor: 6
student: 199
student: 203
student: 225
student: 241
student: 234
student: 195
student: 195
student: 195
student: 237
student: 243
student: 265
student: 194
student: 232
student: 232
student: 232
student: 209
student: 222
student: 222
student: 235
student: 216
student: 216
student: 228
student: 210
student: 210
student: 

210
student: 230
student: 246
student: 255
student: 255
student: 239
student: 239
student: 227
student: 226
student: 226
student: 231
student: 231
student: 220
student: 240
student: 259
student: 221
student: 221


student: 221
student: 221
student: 248
student: 242
student: 242
student: 242
student: 268
student: 214
student: 261
student: 254
student: 254
student: 217
student: 217
student: 211
student: 263
student: 215
student: 215
student: 256
student: 253


student: 208
student: 208
student: 208
student: 238
student: 247
student: 262
student: 262
student: 262
student: 233
student: 200


student: 229
student: 252
student: 204
student: 204
student: 213
student: 213
student: 250
student: 223
student: 245
student: 196
student: 196
student: 196
student: 205
student: 260
student: 244


student: 212


In [15]:
groups = df_log.groupby("E-mail")

In [16]:
mydb = mysql.connector.connect(database="airflow_db", user="db_kali", password="123456789", host="localhost")
mycursor = mydb.cursor()
mycursor.execute("SELECT * FROM students")
myresult = mycursor.fetchall()
emails=[]
for result in myresult:
    emails.append(result[3])

In [17]:
print("\n".join(emails))

amenani@insea.ac.ma
amsanoussilabo@insea.ac.ma
aezzouine@insea.ac.ma 
aajarray@insea.ac.ma
acheddad@insea.ac.ma
aelomairi@insea.ac.ma
asinare@insea.ac.ma
aoussar@insea.ac.ma
ahamdi@insea.ac.ma
aelhadine@insea.ac.ma
cbelkarkor@insea.ac.ma
cbenallalhassani@insea.ac.ma
hchafai@insea.ac.ma
hhoumane@insea.ac.ma
hdah@insea.ac.ma
hlagramez@insea.ac.ma
imourad@insea.ac.ma
ikonkobo@insea.ac.ma
ibelmezouar@insea.ac.ma
kbensatti@insea.ac.ma
yakhallouq@insea.ac.ma
mamallah@insea.ac.ma
magouram@insea.ac.ma
msidibe@insea.ac.ma
nchahboun@insea.ac.ma
rouhmiz@insea.ac.ma
selhairech@insea.ac.ma
sdaouaji@insea.ac.ma
tmajjati@insea.ac.ma
yelhilali@insea.ac.ma
yamaghoss@insea.ac.ma
ysadki@insea.ac.ma
hbounar@insea.ac.ma
sboukili@insea.ac.ma
aelfadili@insea.ac.ma
zjaafari@insea.ac.ma
seaouari@insea.ac.ma
hboudinar@insea.ac.ma
helmouhsine@insea.ac.ma
fbanah@insea.ac.ma
selhasnaoui@insea.ac.ma
abalij@insea.ac.ma
hmehdioui@insea.ac.ma
smellak@insea.ac.ma
ohabiballah@insea.ac.ma
amamansouley@insea.ac.ma
zrhosne

In [18]:

for email in emails:
    name,gender,email,course,major = None,None,email,course,None
    id_student=add_student(name,gender,email,course,major)
    try:
        data = groups.get_group(email).reset_index()
        nb_abs = 0
        #DURATION
        duration=0
        got_first_start=False
        for index, row in data.iterrows():
            start_str = row["Heure d'arrivée"].replace(" à","")
            end_str   = row["Heure de départ"].replace(" à","")
            start = datetime.datetime.strptime(start_str, '%d/%m/%Y %H:%M:%S')
            end = datetime.datetime.strptime(end_str, '%d/%m/%Y %H:%M:%S')
            duration = duration + ((end-start).total_seconds() / 60.0)
            if not got_first_start:
                got_first_start = True
                first_start = start
        if duration > 120:
            duration=120
        #NB LATES  
        if ((first_start-start_course).total_seconds() / 60.0) > 15:
            nb_lates = 1
        else:
            nb_lates = 0
        
        nb_pres = 1
        
        #ABS DURATION
        abs_duration = 120-duration
        
        #DURATION IN INTEGER IN DB!
        duration = int(duration)
        
        #is student
        print("student:",id_student)
        if duration <= 15:
            nb_abs = 1
            nb_lates=0
            nb_pres=0
    
    except KeyError as e:
        print("not found",e)
        nb_abs = 1
        abs_duration = 120
        nb_lates=0
        nb_pres=0
        duration=0
    #insert to data mart now
    try:
        mydb = mysql.connector.connect(database="airflow_db", user="db_kali", password="123456789", host="localhost")
        mycursor = mydb.cursor()
        sql = "INSERT INTO fact_table (id_time,id_prof,id_student,nb_abs,abs_duration,nb_lates,nb_pres,duration) VALUES (%s,%s,%s,%s,%s,%s,%s,%s);"
        val = (id_time,id_prof,id_student,nb_abs,abs_duration,nb_lates,nb_pres,duration)
        mycursor.execute(sql, val)
        mydb.commit()
    except :
        continue

not found 'amenani@insea.ac.ma'
student: 194
not found 'aezzouine@insea.ac.ma '
student: 196
not found 'acheddad@insea.ac.ma'
not found 'aelomairi@insea.ac.ma'
student: 199
student: 200
not found 'ahamdi@insea.ac.ma'
not found 'aelhadine@insea.ac.ma'
student: 203
student: 204
student: 205


not found 'hhoumane@insea.ac.ma'
not found 'hdah@insea.ac.ma'
student: 208
student: 209
student: 210
student: 211
student: 212
student: 213
student: 214
student: 215
student: 216
student: 217
not found 'rouhmiz@insea.ac.ma'
not found

 'selhairech@insea.ac.ma'
student: 220
student: 221
student: 222
student: 223
not found 'ysadki@insea.ac.ma'
student: 225
student: 226
student: 227
student: 228
student: 229
student: 230
student: 231
student: 232
student: 233


student: 234
student: 235
not found 'smellak@insea.ac.ma'
student: 237
student: 238
student: 239
student: 240


student: 241
student: 242


student: 243
student: 244
student: 245
student: 246
student: 247
student: 248
not found 'cbouzouf@insea.ac.ma'
student: 250
not found 'mafarfaoua@insea.ac.ma'
student: 252


student: 253
student: 254
student: 255
student: 256
not found 'hgaoui@insea.ac.ma'
not found 'yaitmalek@insea.ac.ma'
student: 259
student: 260


student: 261
student: 262
student: 263
not found 'szarrouq@insea.ac.ma'
student: 265
not found 'aelkassmi@insea.ac.ma'
not found 'tebouzelmad@insea.ac.ma'
student: 268
not found 'yaitmha@insea.ac.ma'
not found 'aramoudi@insea.ac.ma'
not found 'hhabbouza@insea.ac.ma'


In [19]:
os.system(f'mv "{input_file}" "/home/kali/project/processed/"')

0